Data Anaylysis how much time is spent per ride Based on the following Factors:

- Based on date and time factors such as day of week and time of day
- Based on which station is the starting and / or ending station
- Based on age of the rider at time of the ride
- Based on whether the rider is a member or a casual rider

In [0]:
# Analyze how much time is spent per ride -->  Based on date and time factors such as day of week and time of day

rider_stats_by_day_hour=spark.sql (  '''SELECT rider_id, DAY(started_at) as DAY, extract(HOUR FROM started_at) as HOUR, count(*) as num_rides, AVG( DATEDIFF (second, started_at, ended_at)) * 60 as minutes_spent
    FROM trip_fact
    GROUP by rider_id, DAY(started_at),extract(HOUR FROM started_at)
    Order By rider_id,DAY(started_at),extract(HOUR FROM started_at)'''
    )
rider_stats_by_day_hour.createOrReplaceTempView("rider_stats_by_day_hour")
spark.sql ("Select * from rider_stats_by_day_hour limit 5").show()

+--------+---+----+---------+-------------+
|rider_id|DAY|HOUR|num_rides|minutes_spent|
+--------+---+----+---------+-------------+
|    1000|  2|  10|        1|      95640.0|
|    1000|  2|  17|        1|      53880.0|
|    1000|  2|  23|        1|      31620.0|
|    1000|  3|  14|        1|      84660.0|
|    1000|  6|  19|        1|     170820.0|
+--------+---+----+---------+-------------+



In [0]:
# How Much Time Spent based on which station is the starting and / or ending station
trp_stats_by_stations=spark.sql("select start_station_id,end_station_id,count(distinct trip_id) as trip_count,sum(trip_duration) as timespent,avg(trip_duration) as avg_time from trip_fact group by start_station_id,end_station_id")
trp_stats_by_stations.createOrReplaceTempView("trp_stats_by_stations")
spark.sql("select * from trp_stats_by_stations limit 2").show()

+----------------+--------------+----------+---------+------------------+
|start_station_id|end_station_id|trip_count|timespent|          avg_time|
+----------------+--------------+----------+---------+------------------+
|             600|           600|       114|  37575.0|329.60526315789474|
|    TA1308000022|  TA1308000022|      1294|  51259.0|39.612828438948995|
+----------------+--------------+----------+---------+------------------+



In [0]:
#Analyze how much time is spent per ride
#Based on age of the rider at time of the ride
rider_stats_by_age=spark.sql("select rider_age,count(trip_id) as trp_count,avg(trip_duration) as avg_timespent from trip_fact group by rider_age")

rider_stats_by_age.createOrReplaceTempView("rider_avg_stats_by_age")
spark.sql("select * from rider_avg_stats_by_age ORder By rider_age desc limit 2").show()


+---------+---------+------------------+
|rider_age|trp_count|     avg_timespent|
+---------+---------+------------------+
|       75|       45|              51.4|
|       74|     1768|19.059954751131222|
+---------+---------+------------------+



In [0]:
#Based on whether the rider is a member or a casual rider
rider_stats_by_member_type=spark.sql("select  CASE WHEN is_member=true then 'member' else 'casualrider' end as Member_type,count(trip_id) as trp_count,round(sum(trip_duration)/60 ) as total_timespent_hrs from trip_fact group by  CASE WHEN is_member=true then 'member' else 'casualrider' end ")
rider_stats_by_member_type.createOrReplaceTempView("rider_stats_by_member_type")
spark.sql("select * from rider_stats_by_member_type limit 2").show()

+-----------+---------+-------------------+
|Member_type|trp_count|total_timespent_hrs|
+-----------+---------+-------------------+
|casualrider|   918615|           326582.0|
|     member|  3666306|          1338940.0|
+-----------+---------+-------------------+



In [0]:

# aggregate count of rides and duration by rider, by month
rider_stats_month_year = spark.sql(
    """
    SELECT  MONTH(started_at) as month, YEAR(started_at) as year, count(*) as num_rides, AVG( DATEDIFF (second, started_at, ended_at)) * 60 as minutes_spent
    FROM trip_fact
    GROUP by  MONTH(started_at), YEAR(started_at);
    """
)
rider_stats_month_year.createOrReplaceTempView("rider_stats_month_year")
spark.sql("select * from rider_stats_month_year limit 2").show()

+-----+----+---------+-----------------+
|month|year|num_rides|    minutes_spent|
+-----+----+---------+-----------------+
|    4|2021|   298207|86494.35231231997|
|    5|2021|   450994|95410.06802751256|
+-----+----+---------+-----------------+



Analyze how much money is spent
- Per month, quarter, year
- Per member, based on the age of the rider at account start

In [0]:
#how much money is spent Per month, quarter, year Per Rider

rider_stats_amount=spark.sql ('''
           select r.rider_id,dat.year ,dat.quarter , dat.month ,sum(amount) as Total_Amount, Avg(amount) Avg_Amount
           From
           dim_rider r join payment_fact p on r.rider_Id=p.rider_Id
           join date_dim dat on dat.Id=p.date_id
           
           Group By r.rider_id,dat.year ,dat.quarter , dat.month
           Order By r.rider_id,dat.year ,dat.quarter , dat.month
           ''')
rider_stats_amount.createOrReplaceTempView("rider_stats_amount")
spark.sql("select * from rider_stats_amount limit 5").show()

+--------+----+-------+-----+------------+----------+
|rider_id|year|quarter|month|Total_Amount|Avg_Amount|
+--------+----+-------+-----+------------+----------+
|    1000|2019|      2|    5|         9.0|       9.0|
|    1000|2019|      2|    6|         9.0|       9.0|
|    1000|2019|      3|    7|         9.0|       9.0|
|    1000|2019|      3|    8|         9.0|       9.0|
|    1000|2019|      3|    9|         9.0|       9.0|
+--------+----+-------+-----+------------+----------+



In [0]:
money_spent=spark.sql ('''
           select dat.year ,dat.quarter , dat.month ,sum(amount) as Total_Amount, Avg(amount) Avg_Amount
           From
           dim_rider r join payment_fact p on r.rider_Id=p.rider_Id
           join date_dim dat on dat.id=p.date_id
           
           Group By dat.year ,dat.quarter , dat.month
           Order By dat.year ,dat.quarter , dat.month
           ''')
money_spent.createOrReplaceTempView("money_spent")
spark.sql("select * from money_spent limit 5").show()

+----+-------+-----+------------------+------------------+
|year|quarter|month|      Total_Amount|        Avg_Amount|
+----+-------+-----+------------------+------------------+
|2013|      1|    2|              12.9|              12.9|
|2013|      1|    3| 817.7499999999999|10.095679012345677|
|2013|      2|    4|1672.6500000000003|  9.89733727810651|
|2013|      2|    5|           2716.71|10.290568181818182|
|2013|      2|    6|3775.2999999999997|10.148655913978494|
+----+-------+-----+------------------+------------------+



In [0]:
#Per member, based on the age of the rider at account start

rider_stats_by_age=spark.sql('''select floor (months_between (account_start_Date, birthday) / 12) as rider_age,sum(p.amount) as Total_Amt   From
           dim_rider r join payment_fact p on r.rider_Id=p.rider_Id
           where is_member=true
          Group By floor (months_between (account_start_Date, birthday) / 12)
                     ''')
rider_stats_by_age.createOrReplaceTempView("rider_stats_by_age")
spark.sql("select * from rider_stats_by_age limit 4").show()

+---------+---------+
|rider_age|Total_Amt|
+---------+---------+
|       29| 438219.0|
|       26| 470439.0|
|       65|   8469.0|
|       54|  58365.0|
+---------+---------+



In [0]:
# how many minutes the rider spends on a bike per month
spark.sql ('''
           select rider_id,MONTH(started_at) as Month,count(distinct trip_id) as trip_count, sum(trip_duration) as total_trip_mns 
           ,AVG(COUNT(DISTINCT trip_id)) OVER (PARTITION BY rider_id, MONTH(started_at)) AS avg_monthly_rides
             from trip_fact 
           Group by rider_id,MONTH(started_at)
           limit 4
           ''').show()

+--------+-----+----------+--------------+-----------------+
|rider_id|Month|trip_count|total_trip_mns|avg_monthly_rides|
+--------+-----+----------+--------------+-----------------+
|    1000|    4|         2|          34.0|              2.0|
|    1000|   11|         3|          58.0|              3.0|
|    1003|    6|        18|         296.0|             18.0|
|    1003|   10|        16|         190.0|             16.0|
+--------+-----+----------+--------------+-----------------+



In [0]:
# how many minutes the rider spends on a bike per month
spark.sql ('''
           select t.rider_id,MONTH(started_at) as Month,count(distinct trip_id) as trip_count, sum(trip_duration) as total_trip_mns 
           ,AVG(COUNT(DISTINCT trip_id)) OVER (PARTITION BY t.rider_id, MONTH(started_at)) AS avg_monthly_rides
             from trip_fact t join Dim_rider d on d.rider_id=t.rider_id
           Group by t.rider_id,MONTH(started_at)
           limit 4
           ''').show()

+--------+-----+----------+--------------+-----------------+
|rider_id|Month|trip_count|total_trip_mns|avg_monthly_rides|
+--------+-----+----------+--------------+-----------------+
|    1000|    4|         2|          34.0|              2.0|
|    1000|   11|         3|          58.0|              3.0|
|    1003|    6|        18|         296.0|             18.0|
|    1003|   10|        16|         190.0|             16.0|
+--------+-----+----------+--------------+-----------------+

